# SageMaker Demo Cleanup
- Empties the S3 bucket so it can be deleted by CloudFormation

## UPDATE THE FOLLOWING VALUES FOR THE NOTEBOOK TO WORK CORRECTLY

In [1]:
# Replace with your value for NotebookInstanceName
NotebookInstanceName = "sagemaker-demo-202403130936"

region = "us-west-2"

In [2]:
import boto3

ssm_client = boto3.client("ssm")

In [3]:
# Read value from Parameter Store
def read_from_parameter_store(name):
    response = ssm_client.get_parameter(Name=name, WithDecryption=True)
    return response["Parameter"]["Value"]


# Get the S3 bucket name from Parameter Store
bucket_name = read_from_parameter_store(f"/{NotebookInstanceName}/S3Bucket")
print(f"\tS3 Bucket Name from ParameterStore: {bucket_name}")

	S3 Bucket Name from ParameterStore: sagemaker-demo-202403130936


In [4]:
!aws s3 rm s3://{bucket_name} --recursive

delete: s3://sagemaker-demo-202403130936/data/data-1.html
delete: s3://sagemaker-demo-202403130936/data/data-2.html
delete: s3://sagemaker-demo-202403130936/data/data-3.html
delete: s3://sagemaker-demo-202403130936/index.html
delete: s3://sagemaker-demo-202403130936/data/graph-cluster-2d-visualization.html
delete: s3://sagemaker-demo-202403130936/data/data-6.html
delete: s3://sagemaker-demo-202403130936/data/data-5.html
delete: s3://sagemaker-demo-202403130936/data/graph-cluster-3d-visualization.html
delete: s3://sagemaker-demo-202403130936/data/data-4.html
delete: s3://sagemaker-demo-202403130936/data/pi-data.pkl
delete: s3://sagemaker-demo-202403130936/data/data-7.html


In [5]:
from IPython.display import display, Markdown

markdown_text = f"""
## S3 Bucket has been emptied
Proceed with decommissioning the lab by deleteing the CloudFormation stack

## [CloudFormation Stacks](https://{region}.console.aws.amazon.com/cloudformation/home)
"""

display(Markdown(markdown_text))


## S3 Bucket has been emptied
Proceed with decommissioning the lab by deleteing the CloudFormation stack

## [CloudFormation Stacks](https://us-west-2.console.aws.amazon.com/cloudformation/home)
